## Failed attempt at loading the data into a dynamic table using Flink's Table API

In [ ]:
from pyflink.dataset import ExecutionEnvironment
from pyflink.table import TableConfig, StreamTableEnvironment

exec_env = ExecutionEnvironment.get_execution_environment()
exec_env.set_parallelism(1)
t_config = TableConfig()
t_env = StreamTableEnvironment.create(exec_env, t_config)

# specify input format as table rows
twitter_table = """
    CREATE TABLE tweets (
        data ROW<text STRING, public_metrics ROW<retweet_count INT, reply_count INT, like_count INT, quote_count INT>, 
                 author_id STRING, id STRING, created_at STRING, source STRING, lang STRING>,
        includes ROW<users ARRAY<ROW<protected BOOLEAN, id STRING, name STRING, created_at STRING, 
                     public_metrics ROW<followers_count INT, following_count INT, tweet_count INT, listed_count INT>, 
                                        username STRING>>>,
        matching_rules ARRAY<ROW<id INT, tag STRING>>,
        kafka_ts TIMESTAMP(3) METADATA FROM 'timestamp'        
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'twitter-stream',
        'properties.bootstrap.servers' = 'localhost:9092',
        'properties.group.id' = 'twitter_consumer'
        'format' = 'json'   
        'json.fail-on-missing-field' = 'false',
        'json.ignore-parse-errors' = 'true'     
    )
"""

t_env.execute_sql(twitter_table)


'''
Exemplary Tweet Format: 
{"data": {"text": "Polsek Kakas Cegah Covid-19 https://t.co/ADjEgpt7bC",
          "public_metrics": {"retweet_count": 0, "reply_count": 0, "like_count": 0, "quote_count": 0},
          "author_id": "1367839185764151302", "id": "1378275866279469059", "created_at": "2021-04-03T09:19:08.000Z",
          "source": "Twitter for Android", "lang": "in"},
 "includes": {"users": [{"protected": false, "id": "1367839185764151302", "name": "Nathan Pareda",
                         "created_at": "2021-03-05T14:07:56.000Z",
                         "public_metrics": {"followers_count": 0, "following_count": 0, "tweet_count": 557,
                                            "listed_count": 0}, "username": "NathanPareda"}]},
 "matching_rules": [{"id": 1378112825051246596, "tag": "coronavirus"}]}
'''